### Modeling.

Here comes the really fun step: modeling! For this step, you'll consider the types of models that would be appropriate to use given the features in your dataset and then dive right in and build two or three models. Please note: in addition to considering different algorithm types in your model selection, be sure to also consider applying model hyperparameter tuning operations. 

In [21]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import Preprocessing as data

In [22]:
Xtest = data.Xtest
Xtrain = data.Xtrain
ytest = data.ytest
ytrain = data.ytrain

features = data.features

#### Start with a logistic regression

In [23]:
from sklearn.linear_model import LogisticRegressionCV

logreg = LogisticRegressionCV(cv=5)
logreg.fit(Xtrain,ytrain)
probs = logreg.predict_proba(Xtest)
print('Logistic Regression Score is :',round(logreg.score(Xtest,ytest),2))

logreg_feat = logreg.coef_

Logistic Regression Score is : 0.92


In [24]:
features

['hospital_id',
 'age',
 'bmi',
 'ethnicity',
 'elective_surgery',
 'has_h1',
 'icu_admit_source',
 'icu_type',
 'apache_3j_bodysystem',
 'apache_3j_diagnosis',
 'apache_2_diagnosis',
 'lactate',
 'ACR',
 'h1_temp_min']